In [1]:
import torch

from dpp_nets.my_torch.simulator import SimKDPPDeepSet
from dpp_nets.my_torch.simulator import SimFilter
from dpp_nets.helper.plotting import plot_floats, plot_dict

import torch
import torch.nn as nn
from torch.autograd import Variable
from dpp_nets.my_torch.linalg import custom_decomp
from dpp_nets.my_torch.DPP import DPP
from dpp_nets.my_torch.utilities import compute_baseline

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

from dpp_nets.my_torch.linalg import custom_decomp
from dpp_nets.my_torch.DPP import DPP
from dpp_nets.my_torch.utilities import compute_baseline
from dpp_nets.my_torch.utilities import pad_tensor

import dpp_nets.my_torch

import numpy as np 
from collections import defaultdict


In [2]:
from dpp_nets.layers.layers import *

In [60]:
class SimKDPPDeepSet(object):

    def __init__(self, network_params, dtype):

        # Customizable parameters
        self.set_size = set_size = network_params['set_size'] # 40
        self.n_clusters = n_clusters = network_params['n_clusters'] # 
        self.dtype = dtype

        # Fixed parameters
        self.kernel_in = kernel_in = 100
        self.kernel_h = kernel_h = 500
        self.kernel_out = kernel_out = 100

        self.pred_in = pred_in = 50 # kernel_in / 2
        self.pred_h = pred_h = 500
        self.pred_out = pred_out = 100
        
        # 2-Hidden-Layer Networks 
        self.kernel_net = KernelFixed(pred_in, kernel_h, kernel_out)
        self.kernel_net.type(self.dtype)

        self.pred_net = DeepSetPred(pred_in, pred_h, pred_out, pred_in)
        self.pred_net.type(self.dtype)

        # Data
        np.random.seed(0)
        self.means = dtype(np.random.randint(-10,10,[n_clusters, int(pred_in)]).astype("float"))
        self.saved_subsets = None
        self.saved_losses = None
        self.saved_baselines = None

        # 
        self.criterion = nn.MSELoss()

        # Record loss
        self.loss_dict = defaultdict(list)
        self.prec_dict = defaultdict(list)
        self.rec_dict = defaultdict(list)
        self.ssize_dict = defaultdict(list)

        # Useful intermediate variables 
        self.embedding = None
        self.subset = None
        self.pick = None
        self.pred = None


    def generate(self, batch_size):
        """sdf"
        Arguments:
        means: Probs best to make this an attribute of the class, 
        so that repeated training works with the same data distribution.


        """
        batch_size = batch_size
        n_clusters = self.n_clusters
        set_size = self.set_size
        embd_dim = self.pred_in
        dtype = self.dtype
        means = self.means

        # Generate index
        index = torch.cat([torch.arange(0, float(n_clusters)).expand(batch_size, n_clusters).long(), 
                          torch.multinomial(torch.ones(batch_size, n_clusters), set_size - n_clusters, replacement=True)]
                         ,dim=1)
        index = index.t()[torch.randperm(set_size)].t().contiguous()

        # Generate words, context, target
        words = dtype(torch.normal(means.index_select(0,index.view(index.numel()))).view(batch_size, set_size, embd_dim))
        context = dtype(words.sum(1).expand_as(words))

        target = torch.sin(words.mean(1)).squeeze()

        return words, context, target, index 

    def train(self, train_iter, batch_size, lr, alpha_iter=1, baseline=True, reg=0, reg_mean=0):
        """
        Training the model. 
        Doesn't use the forward pass as want to sample repeatedly!

        """
        set_size = self.set_size
        n_clusters = self.n_clusters
        kernel_in = self.kernel_in
        kernel_out = self.kernel_out
        embd_dim = self.pred_in
        dtype = self.dtype

        loss_log = 100
        params = list(self.kernel_net.parameters()) + list(self.pred_net.parameters())
        optimizer = optim.Adam(params, lr=lr)

        self.loss_dict.clear()
        self.ssize_dict
      
        sample = SampleFixed(alpha_iter)

        for t in range(train_iter):
            
            words, context, target, index = self.generate(batch_size)
            words = Variable(words)
            target = Variable(target)
            
            kernel = self.kernel_net(words)
            word_picks = sample(kernel, words)
            pred = self.pred_net(word_picks)
            action_list = sample.saved_subsets
            exp_sizes = sample.exp_sizes

            loss = custom_backprop(reg, reg_mean)(pred, target, action_list, exp_sizes)
            
            ssizes = [sample.data.sum() for example in action_list for sample in example]
            ssize = sum(ssizes) / len(ssizes)

            optimizer.step()
            optimizer.zero_grad()

            self.loss_dict[t].append(loss.data[0])
            self.ssize_dict[t].append(ssize)

            if not ((t + 1) % loss_log):
                print("Loss at it ", t+1, " is: ", loss.data[0])

    def evaluate(self, test_iter):
        
        set_size = self.set_size
        n_clusters = self.n_clusters
        kernel_in = self.kernel_in
        kernel_out = self.kernel_out
        embd_dim = self.pred_in
        dtype = self.dtype

        sample = SampleFixed(alpha_iter)
        
        cum_loss = 0. 
        cum_ssize = 0.

        for t in range(test_iter):
            words, context, target, index = self.generate(batch_size)
            words = Variable(words, volatile=True)
            target = Variable(target, volatile=True)
            kernel = self.kernel_net(words)
            word_picks = sample(kernel, words)
            pred = self.pred_net(word_picks)
            target = target.unsqueeze(1).expand_as(pred)
            action_list = sample.saved_subsets
            exp_sizes = sample.exp_sizes
            
            loss = nn.MSELoss()(pred, target)
            cum_loss += loss.data[0]
            
            
            ssizes = [sample.data.sum() for example in action_list for sample in example]
            ssize = sum(ssizes) / len(ssizes)
            cum_ssize += ssize
            
        print("Average Loss is:", cum_loss / test_iter)
        print("Average Set Size is:", cum_ssize / test_iter)


In [68]:
network_params = {'set_size': 20, 'n_clusters': 15}
dtype = torch.DoubleTensor
train_iter = 300
batch_size = 10
lr = 1e-4
alpha_iter=4
baseline=True

In [70]:
# Simulation with 20
reg=0.1
reg_mean=20
torch.manual_seed(10)
sim20 = SimKDPPDeepSet(network_params, dtype)
sim20.train(1000, batch_size, lr, alpha_iter, baseline, reg, reg_mean)
sim20.evaluate(500)

In [71]:
sim20.train(1000, batch_size, lr, alpha_iter, baseline, reg, reg_mean)
sim20.evaluate(500)
plot_floats(sim20.loss_dict,50)
plot_floats(sim20.ssize_dict,50)

Loss at it  100  is:  0.1755370034433319
Loss at it  200  is:  0.17093300190298236
Loss at it  300  is:  0.1763580477392184
Loss at it  400  is:  0.1297410110063765
Loss at it  500  is:  0.11246684998262738
Loss at it  600  is:  0.08764125453645899
Loss at it  700  is:  0.07516024789664907
Loss at it  800  is:  0.06069978859970976
Loss at it  900  is:  0.053976762732496734
Loss at it  1000  is:  0.055291232107124645
Average Loss is: 0.054701205642064786
Average Set Size is: 19.8374


In [72]:
# Simulation with 15
reg=0.1
reg_mean=15
torch.manual_seed(10)
sim15 = SimKDPPDeepSet(network_params, dtype)
sim15.train(1000, batch_size, lr, alpha_iter, baseline, reg, reg_mean)
sim15.evaluate(500)

Loss at it  100  is:  0.1981353982956308
Loss at it  200  is:  0.1882415775646287
Loss at it  300  is:  0.19322193633764267
Loss at it  400  is:  0.1739071667338106
Loss at it  500  is:  0.20731637771426475
Loss at it  600  is:  0.17668607345866602
Loss at it  700  is:  0.2095282350300861
Loss at it  800  is:  0.1415895211387435
Loss at it  900  is:  0.1335014511240102
Loss at it  1000  is:  0.1402187680474526
Average Loss is: 0.14020579583455836
Average Set Size is: 15.266549999999993


In [73]:
# Simulation with 10
reg=0.1
reg_mean=10
torch.manual_seed(10)
sim10 = SimKDPPDeepSet(network_params, dtype)
sim10.train(1000, batch_size, lr, alpha_iter, baseline, reg, reg_mean)
sim10.evaluate(500)

Loss at it  100  is:  0.19781468929543708
Loss at it  200  is:  0.21891356983351115
Loss at it  300  is:  0.18709956410294123
Loss at it  400  is:  0.19181267800507157
Loss at it  500  is:  0.21664752443260968
Loss at it  600  is:  0.2071191988493325
Loss at it  700  is:  0.19925072163100702
Loss at it  800  is:  0.18404080067923678
Loss at it  900  is:  0.15778632437412848
Loss at it  1000  is:  0.1782115126004704
Average Loss is: 0.17305462747333808
Average Set Size is: 9.837050000000005


In [12]:
# Simulation with 5
reg=100
reg_mean=5
torch.manual_seed(10)
sim5 = SimKDPPDeepSet(network_params, dtype)
sim5.train(train_iter, batch_size, lr, alpha_iter, baseline, reg, reg_mean)
sim5.evaluate(500)

Loss at it  100  is:  0.2873951549034986
Loss at it  200  is:  0.2947912424768284
Average Loss is: 0.29981243191328744
Average Set Size is: 16.5232


In [19]:
sim5.train(train_iter, batch_size, lr, alpha_iter, baseline, 10000, 5)



Loss at it  100  is:  0.30551239271019875
Loss at it  200  is:  0.26461144107922957


In [22]:
sim5.evaluate(100)

Average Loss is: 0.3004675358128807
Average Set Size is: 20.812750000000005


In [9]:
sim

40

In [ ]:
sim.loss_dict
